In [5]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [6]:
%%sql
SELECT 
name,
`type`
FROM sqlite_master
WHERE `type` IN ("table","view");

* sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [7]:
%%sql
SELECT *
FROM artist
LIMIT 3;

* sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith


# Question 1
Which genres sell the most and which artist we should purchase for the store

In [4]:
import pandas as pd 

artist = {\
    'Artist':\
    ['Regal','Red Tone','Meteor and the Girls','Slim Jim Bites'],\
    'Genre':\
    ['Hip-Hop','Punk','Pop','Blues']\
}


pd.DataFrame(artist)

,Artist,Genre
0,Regal,Hip-Hop
1,Red Tone,Punk
2,Meteor and the Girls,Pop
3,Slim Jim Bites,Blues


In [27]:
%%sql
WITH TOTAL as (
    SELECT SUM(quantity) as tot
    FROM invoice_line
)

SELECT 
    genre.name,
    SUM(invoice_line.quantity) ,
    ROUND(SUM(invoice_line.quantity)/CAST((SELECT * FROM TOTAL) AS REAL),2)
FROM genre 
INNER JOIN track ON genre.genre_id = track.genre_id
INNER JOIN invoice_line ON invoice_line.track_id = track.track_id
GROUP BY 1
ORDER BY 2 DESC;





* sqlite:///chinook.db
Done.


name,SUM(invoice_line.quantity),"ROUND(SUM(invoice_line.quantity)/CAST((SELECT * FROM TOTAL) AS REAL),2)"
Rock,2635,0.55
Metal,619,0.13
Alternative & Punk,492,0.1
Latin,167,0.04
R&B/Soul,159,0.03
Blues,124,0.03
Jazz,121,0.03
Alternative,117,0.02
Easy Listening,74,0.02
Pop,63,0.01


# Question 2

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.


In [34]:
%%sql
WITH supp AS (SELECT 
    customer.customer_id,
    customer.support_rep_id,
    ROUND(SUM(invoice.total),2) user_purchased
FROM invoice INNER JOIN customer ON invoice.customer_id = customer.customer_id
GROUP BY 1,2)

SELECT 
    employee.employee_id,
    employee.title,
    employee.country,
    ROUND(SUM(supp.user_purchased),2)
FROM employee INNER JOIN supp ON employee.employee_id = supp.support_rep_id
GROUP BY 1;

* sqlite:///chinook.db
Done.


employee_id,title,country,"ROUND(SUM(supp.user_purchased),2)"
3,Sales Support Agent,Canada,1731.51
4,Sales Support Agent,Canada,1584.0
5,Sales Support Agent,Canada,1393.92


# Question 3

In [59]:
%%sql
WITH customer_total AS ( SELECT
    customer.country,
    COUNT(customer.country) as total_customer
FROM customer
GROUP BY 1) ,
    country_total AS (SELECT  customer_total.* ,
        CASE WHEN total_customer = 1 THEN 1
            ELSE 0 END AS other
    FROM customer_total)

SELECT 
    invoice.billing_country `Country`,
    COUNT(invoice.customer_id) `Total No. of Customers`,
    ROUND(SUM(invoice.total),2) `Total Value of Sales`,
    ROUND(SUM(invoice.total) / COUNT(invoice.customer_id),2) `Average Value of Sales`,
    ROUND(AVG(invoice.total),2) `Average Order Value`
FROM invoice INNER JOIN country_total ON invoice.billing_country = country_total.country
GROUP BY 1
ORDER BY country_total.other ASC;


* sqlite:///chinook.db
Done.


Country,Total No. of Customers,Total Value of Sales,Average Value of Sales,Average Order Value
Brazil,61,427.68,7.01,7.01
Canada,76,535.59,7.05,7.05
Czech Republic,30,273.24,9.11,9.11
France,50,389.07,7.78,7.78
Germany,41,334.62,8.16,8.16
India,21,183.15,8.72,8.72
Portugal,29,185.13,6.38,6.38
USA,131,1040.49,7.94,7.94
United Kingdom,28,245.52,8.77,8.77
Argentina,5,39.6,7.92,7.92
